In [1]:
import os
from datetime import datetime, timedelta
from google.cloud import storage
import pymysql
import pymssql
import pandas as pd
from pandas_gbq import to_gbq
from google.cloud import bigquery
import utils.bdds as bdds

current_directory = os.getcwd()
print(current_directory)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = bdds.GetGCP()

ModuleNotFoundError: No module named 'utils'

In [25]:
def getMysqlData(fecha):
    host = '172.26.200.11'
    user = 'root'
    password = 'admininformatica'
    database = 'dw'

    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        db=database,
        charset="utf8mb4",
        )

    # Define the SQL query to select data from a table
    sql_libras = f"""Select 
        fechaproduccion,  year(fechaproduccion) year, month(fechaproduccion) month, 
        case WHEN (WEEK(fechaproduccion,3) >= 52 AND DAYOFYEAR(fechaproduccion) <= 3 ) THEN 1 ELSE WEEK(fechaproduccion,3) END as week,
        articulo, turno,
        SUM(libras) libras, SUM(cantidad) unidades_solicitadas, SUM(entregado) unidades_entregadas
          from (
    SELECT
    pro_orden.fechaproduccion,
    pro_orden.articulo,
    pro_orden.codigo as turno,
    0 as libras,
    SUM(pro_orden.cantidad) cantidad,
    SUM(pro_orden.entregado) entregado
    FROM
    dw.pro_orden
    where
    (fechaproduccion) > '{fecha}' 
    and Estado = 'Cerrada'
    group by
    pro_orden.fechaproduccion, pro_orden.articulo,pro_orden.codigo
    UNION ALL
    SELECT
        fechaproduccion,
        producto as articulo,
        turno,
        libras,
        0 as cantidad,
        0 as entregado
    FROM
        dw.pro_librasmojadas
    WHERE
            (`pro_librasmojadas`.`fechaproduccion`) > '{fecha}' 
    ) pro
    group by fechaproduccion, articulo,turno"""

    try:
        # Use Pandas to read data from MySQL into a DataFrame
        result = pd.read_sql(sql_libras, connection)

    except Exception as e:
        print("Error:", str(e))

    finally:
        # Close the database connection when done
        connection.close()
        return result

In [21]:
# Test 
#Probar si funciona la funcion
#loaded_data_produccion = getMysqlData(date_value)
#loaded_data_produccion

C:\Users\wicho\AppData\Local\Temp\ipykernel_4496\2734859263.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(sql_libras, connection)


In [131]:
# Test 
# Por si se necesita guardar localmente
#save to csv file
#loaded_data_produccion.to_csv(current_directory + "\\file.csv", index=False)

In [3]:
# Specify your BigQuery project_id, dataset_id, and table_id
project_id = 'digital-bonfire-344816'
dataset_id = 'Produccion'
table_id = 'ResumenProduccion'

# Initialize a BigQuery client
client = bigquery.Client()

In [4]:
#Borrar los ultimos 60 dias de informacion de BigQUery
sql_delete = f"""DELETE FROM {project_id}.{dataset_id}.{table_id} 
        WHERE
        fechaproduccion >= DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY);"""

# Execute the query
job = client.query(sql_delete)

# Wait for the query to complete
job.result()

print(f"Deleted {job.num_dml_affected_rows} rows.")

NotFound: 404 Not found: Dataset digital-bonfire-344816:Produccion was not found in location US

Location: US
Job ID: 95eb9f0a-693a-4ef5-9fe3-4cadb6d322bc


In [28]:
# Obtener el ultimo dia con informacion en BigQuery
# SQL query to retrieve MAX(date) to update data from that date
sql_query = f"""
    SELECT MAX(fechaproduccion) as LastDate FROM `digital-bonfire-344816.Produccion.ResumenProduccion` 
LIMIT 100"""
client = bigquery.Client()

# Execute the query
query_job = client.query(sql_query)
results = query_job.result()

# Extract the date value from the result
for row in results:
    date_value = row.LastDate

print(date_value.strftime('%Y-%m-%d'))

2023-08-16


In [29]:
loaded_data_produccion = None
#loaded_data_produccion = getMysqlData('2019-12-31')
loaded_data_produccion = getMysqlData(date_value.strftime('%Y-%m-%d'))

#change fechaprodduccion to datetime column type
loaded_data_produccion['fechaproduccion'] = pd.to_datetime(loaded_data_produccion['fechaproduccion'])

C:\Users\wicho\AppData\Local\Temp\ipykernel_4496\4006146411.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(sql_libras, connection)


In [30]:
loaded_data_produccion.head()

,fechaproduccion,year,month,week,articulo,turno,libras,unidades_solicitadas,unidades_entregadas
0,2023-08-17,2023,8,33,amapola fondant C17,FAB2DECORA,0.000000,2.0,2.0
1,2023-08-17,2023,8,33,amapola naranja fondant C23A,FAB2DECORA,0.000000,1.0,1.0
2,2023-08-17,2023,8,33,baguette 4.5,FAB3MOJADO,127.944539,940.0,940.0
3,2023-08-17,2023,8,33,baguette 6.0,FAB3MOJADO,9.999200,50.0,50.0
4,2023-08-17,2023,8,33,baguette corteza,ALM1MOJADO,0.000000,2.0,2.0


In [31]:
loaded_data_produccion.tail()

,fechaproduccion,year,month,week,articulo,turno,libras,unidades_solicitadas,unidades_entregadas
88830,2023-10-16,2023,10,42,Xeca doble sin aniz,FAB3MOJADO,34.672728,0.0,0.0
88831,2023-10-16,2023,10,42,Xeca jalea,FAB3MOJADO,9.952214,0.0,0.0
88832,2023-10-16,2023,10,42,xeca jalea bolsa,FAB3MOJADO,0.000000,0.0,0.0
88833,2023-10-16,2023,10,42,Xeca jamon y queso,FAB3MOJADO,9.432259,0.0,0.0
88834,2023-10-16,2023,10,42,Xeca sin aniz,FAB3MOJADO,13.567612,0.0,0.0


In [32]:
#Cargar desde un dia despues de la ultima fecha cargada
# Write the DataFrame to a BigQuery table
to_gbq(loaded_data_produccion, destination_table=f'{project_id}.{dataset_id}.{table_id}', project_id=project_id, if_exists='append')
print("Loaded data to table {}".format(table_id))

100%|██████████| 1/1 [00:00<?, ?it/s]

Loaded data to table ResumenProduccion
